In [9]:
import os
from dotenv import load_dotenv
# 올바른 임포트 경로로 수정
from langchain_community.document_loaders import PyMuPDFLoader # PyPDFLoader 대신 사용
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_classic.retrievers.multi_query import MultiQueryRetriever

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

# 환경 변수 설정
os.environ['OPENAI_API_KEY'] = openai_api_key

# Loader: 에러가 자주 나는 PyPDFLoader 대신 PyMuPDFLoader 사용
loader = PyMuPDFLoader("sample.pdf")

# 데이터를 불러오고 분할
# load_and_split()에 splitter를 인자로 줄 수 있습니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
pages = loader.load_and_split(text_splitter=text_splitter)

texts = text_splitter.split_documents(pages)

# 결과 확인
print(f"총 {len(pages)}개의 페이지(청크)가 로드되었습니다.")
if pages:
    print("첫 번째 페이지 내용 일부:", pages[0].page_content[:100])

총 7개의 페이지(청크)가 로드되었습니다.
첫 번째 페이지 내용 일부: 에코프로 공급물량 신청 사이트?…"사
기입니다"
"60세 이상 고용률 높이면 잠재성장률
증가"…노동시장 유연성 전제돼야
'오염물질·온실가스 무배출' 수소차 첫
등록 5년 만에 10


In [ ]:
# Embedding
embeddings_model = OpenAIEmbeddings()

#load it into Chroma
vectorstore = Chroma.from_documents(texts,embeddings_model)

chroma_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 1, 'fetch_k': 4}
)


## BM25
- 전통적 정보검색 모델, TF-IDF 와 유사한 방식을 동작
- 사용자 입력 질의와 문서간 관련성을 점수화하여 문서 순위 매김
- 문서 내 검색단어의 빈도와, 문서 길이를 고려, 과도하게 긴 문서가 높은 문서를 받지 않도록 함.
- 주요 특징
    - 단어 빈도: 특정 단어가 얼마나 자주 등장하는 지 계산
    - 역문서 빈도: 특정 단어가 전체 문서 집합에서 얼마나 흔하지 않은지 측정. 흔하지 않은 단어일수록 (그 문서에만 나타날수록) 중요한 정보로 간주
    - 문서 길이 정규화: 문서 길이 보정. 너무 긴 문서는 불리하게, 짧은 문서는 유리하게 가중치 조절.  문서 길이 길면, 길다는 이유로 단어가 자주 등장하여 관련성과 무관하게 점수 높게 받는 현상 방지

$$score(D, Q) = \sum_{q \in Q} \text{IDF}(q) \cdot \frac{f(q, D) \cdot (k_1 + 1)}{f(q, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})}$$